In [8]:
#pip install yfinance

In [12]:
####################################################
## Fetch data from yfinance
## 3-year daily data for Coca-Cola, SPY, Pepsi, and USD index
import datetime
import yfinance as yf
import numpy as np
import pandas as pd

end1 = datetime.date(2024, 12, 31)
start1 = end1 - pd.Timedelta(days = 365 * 3)

nifty = yf.download("^NSEI", start = start1, end = end1, progress = False)
ril = yf.download("RELIANCE.NS", start = start1, end = end1, progress = False)
lt = yf.download("LT.NS", start = start1, end = end1, progress = False)
tcs = yf.download("TCS.NS", start = start1, end = end1, progress = False)
inr = yf.download("INR=X", start = start1, end = end1, progress = False)

####################################################
## Calculate log returns for the period based on Adj Close prices

nifty['nfity'] = np.log(nifty['Adj Close'] / nifty['Adj Close'].shift(1))
ril['ril'] = np.log(ril['Adj Close'] / ril['Adj Close'].shift(1))
lt['lt'] = np.log(lt['Adj Close'] / lt['Adj Close'].shift(1))
tcs['tcs'] = np.log(tcs['Adj Close'] / tcs['Adj Close'].shift(1))
inr['inr'] = np.log(inr['Adj Close'] / inr['Adj Close'].shift(1))

c:\Users\Patil\anaconda3\envs\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\Patil\anaconda3\envs\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\Patil\anaconda3\envs\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\Patil\anaconda3\envs\myenv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated a

In [16]:
####################################################
## Create a dataframe with X's (spy, pep, usdx) and Y (ko)

df = pd.concat([nifty['nfity'], ril['ril'], 
                lt['lt'], tcs['tcs'],inr['inr']], axis = 1).dropna()

## Save the csv file. Good practice to save data files after initial processing
df.to_csv("D:\\SCMA 632\\Py\\Project_4\\G7\\data_lin_regression.csv")

####################################################

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 518 entries, 2022-01-04 to 2024-02-08
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   nfity   518 non-null    float64
 1   ril     518 non-null    float64
 2   lt      518 non-null    float64
 3   tcs     518 non-null    float64
 4   inr     518 non-null    float64
dtypes: float64(5)
memory usage: 24.3 KB


In [17]:
df.corr()

,nfity,ril,lt,tcs,inr
nfity,1.000000,0.658137,0.635742,0.604942,-0.129605
ril,0.658137,1.000000,0.340902,0.300392,-0.074983
lt,0.635742,0.340902,1.000000,0.399671,-0.050535
tcs,0.604942,0.300392,0.399671,1.000000,-0.070599
inr,-0.129605,-0.074983,-0.050535,-0.070599,1.000000


In [20]:
#pip install statsmodels

In [36]:
df.tail()

,nfity,ril,lt,tcs,inr
Date,,,,,
2024-02-02,0.007180,0.021531,-0.006481,0.028683,-0.001518
2024-02-05,-0.003764,-0.012894,-0.010212,0.001763,0.000864
2024-02-06,0.007217,-0.007831,0.024388,0.040132,0.001257
2024-02-07,0.000050,0.010000,-0.008667,-0.012799,-0.000398
2024-02-08,-0.011544,-0.001839,-0.028172,0.009566,-0.001405


In [37]:
df1 = pd.read_csv('D:\\SCMA 632\\Py\\Project_4\\G7\\data_lin_regression.csv')

In [39]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518 entries, 0 to 517
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    518 non-null    object 
 1   nfity   518 non-null    float64
 2   ril     518 non-null    float64
 3   lt      518 non-null    float64
 4   tcs     518 non-null    float64
 5   inr     518 non-null    float64
dtypes: float64(5), object(1)
memory usage: 24.4+ KB


In [49]:
from statsmodels.api import OLS

# Define the dependent and independent variables
y = df['nfity']

# Add a constant term (intercept)
df['const'] = 1

X = df[['const','ril', 'tcs', 'lt','inr']]

# Fit the OLS regression
model = OLS(y, X).fit()

# Print the summary of the regression results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  nfity   R-squared:                       0.717
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     324.9
Date:                Thu, 08 Feb 2024   Prob (F-statistic):          4.43e-139
Time:                        15:27:11   Log-Likelihood:                 2039.0
No. Observations:                 518   AIC:                            -4068.
Df Residuals:                     513   BIC:                            -4047.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.092e-05      0.000      0.243      0.8